In [170]:
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Kman9\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Kman9\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [171]:
import pandas as pd
import numpy as np

In [172]:
# read all data
ph = pd.read_csv('data/share-of-deaths-homicides.csv')
emis = pd.read_csv('data/GCB2022v27_MtCO2_flat.csv')
gdp = pd.read_csv('data/gdp.csv')


In [173]:
# Standardize the column names
ph.rename(columns={'Entity' : 'Country'}, inplace=True)
ph.rename(columns={'Deaths - Interpersonal violence - Sex: Both - Age: All Ages (Percent)' : 'Homicide Rate'}, inplace=True)
gdp.rename(columns={'Country Name' : 'Country'}, inplace=True)

In [174]:
# drop unwanted columns
emis = emis.drop(columns = ['Other'])
emis = emis.drop(columns=['ISO 3166-1 alpha-3'])
ph = ph.drop(columns=['Code'])
gdp = gdp.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code'])

In [175]:
#debucketizing gdp
melted_gdp = gdp.melt(id_vars=['Country'], var_name="Year", value_name='GDP per capita')
#melted_gdp = melted_gdp.dropna()
melted_gdp = melted_gdp.sort_values(["Country","Year"])
melted_gdp = melted_gdp.reset_index(drop=True)
melted_gdp['Year'] = melted_gdp['Year'].astype(int)
melted_gdp['GDP per capita'] = melted_gdp['GDP per capita'].replace(np.nan,0)

melted_gdp[:5]
melted_gdp.isnull().sum()


Country           0
Year              0
GDP per capita    0
dtype: int64

In [176]:
# drop rows with year < 1990 and year > 2019
emis = emis[emis['Year'] >= 1990]
emis = emis[emis['Year'] <= 2019]
#make sure debucketized gdp works first
gdp = melted_gdp[melted_gdp['Year'] >= 1990]
gdp = melted_gdp[melted_gdp['Year'] <= 2019]
emis[:5]

,Country,Year,Total,Coal,Oil,Gas,Cement,Flaring,Per Capita
240,Afghanistan,1990,2.024326,0.278464,1.271408,0.403040,0.045766,0.025648,0.189281
241,Afghanistan,1991,1.914301,0.249627,1.204085,0.389125,0.045766,0.025697,0.178155
242,Afghanistan,1992,1.482054,0.021984,1.029584,0.362736,0.045766,0.021984,0.122916
243,Afghanistan,1993,1.486943,0.018320,1.047904,0.351744,0.046991,0.021984,0.106182
244,Afghanistan,1994,1.453829,0.014693,1.032171,0.337935,0.046991,0.022039,0.094065


In [177]:
# remove NaN values
emis.isnull().sum()

c = emis.groupby('Country')

c.apply(lambda x: x.isnull().sum())


,Country,Year,Total,Coal,Oil,Gas,Cement,Flaring,Per Capita
Country,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0
Albania,0,0,0,0,0,0,0,0,0
Algeria,0,0,0,0,0,0,0,0,0
Andorra,0,0,0,0,0,0,0,0,0
Angola,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
Viet Nam,0,0,0,0,0,0,0,0,0
Wallis and Futuna Islands,0,0,0,0,0,0,11,0,0
Yemen,0,0,0,0,0,0,0,0,0


In [178]:
# set year column as index
emis.set_index('Year', inplace=True)

# fill missing values with previous and forward 5-year averages
emis.fillna(emis.rolling(window=11, min_periods=1).mean().shift(1).fillna(method='bfill'), inplace=True)

# reset index
emis.reset_index(inplace=True)

emis.isnull().sum()

C:\Users\Kman9\AppData\Local\Temp\ipykernel_1628\1088784760.py:5: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Country'], dtype='object')
  emis.fillna(emis.rolling(window=11, min_periods=1).mean().shift(1).fillna(method='bfill'), inplace=True)


Year          0
Country       0
Total         0
Coal          0
Oil           0
Gas           0
Cement        0
Flaring       0
Per Capita    0
dtype: int64

In [179]:
# merging tables
merged = pd.merge(ph, emis, how='inner', on=['Country', 'Year'])
merged = pd.merge(merged, gdp, how="left", on=['Country','Year'])
merged[:25]

,Country,Year,Homicide Rate,Total,Coal,Oil,Gas,Cement,Flaring,Per Capita,GDP per capita
0,Afghanistan,1990,0.84,2.024326,0.278464,1.271408,0.403040,0.045766,0.025648,0.189281,0.000000
1,Afghanistan,1991,1.04,1.914301,0.249627,1.204085,0.389125,0.045766,0.025697,0.178155,0.000000
2,Afghanistan,1992,1.10,1.482054,0.021984,1.029584,0.362736,0.045766,0.021984,0.122916,0.000000
3,Afghanistan,1993,1.15,1.486943,0.018320,1.047904,0.351744,0.046991,0.021984,0.106182,0.000000
4,Afghanistan,1994,1.18,1.453829,0.014693,1.032171,0.337935,0.046991,0.022039,0.094065,0.000000
5,Afghanistan,1995,1.20,1.417327,0.014656,1.011264,0.322432,0.046991,0.021984,0.086323,0.000000
6,Afghanistan,1996,1.33,1.370104,0.007328,0.985616,0.307776,0.047400,0.021984,0.080092,0.000000
7,Afghanistan,1997,1.17,1.304152,0.003664,0.948976,0.282128,0.047400,0.021984,0.073313,0.000000
8,Afghanistan,1998,1.13,1.278504,0.003664,0.941648,0.263808,0.047400,0.021984,0.069134,0.000000
9,Afghanistan,1999,1.13,1.091640,0.003664,0.776768,0.241824,0.047400,0.021984,0.056671,0.000000


In [180]:
# checking for duplicates
duplicates = merged.duplicated(subset=['Country', 'Year'], keep=False)

# print resulting dataframe
# if there are no duplicates, the dataframe will be empty
print(merged[duplicates])

Empty DataFrame
Columns: [Country, Year, Homicide Rate, Total, Coal, Oil, Gas, Cement, Flaring, Per Capita, GDP per capita]
Index: []


In [181]:
merged.describe()

,Year,Homicide Rate,Total,Coal,Oil,Gas,Cement,Flaring,Per Capita,GDP per capita
count,5610.000000,5610.000000,5610.000000,5610.000000,5610.000000,5610.000000,5610.000000,5610.000000,5610.000000,4950.000000
mean,2004.500000,1.095742,120.147593,50.909146,39.571093,22.193680,5.057668,1.462085,4.545368,10250.963517
std,8.656213,1.678670,543.209645,381.784589,105.091306,68.811459,37.316321,4.788171,6.223573,16763.887997
min,1990.000000,0.040000,0.000000,0.000000,0.003664,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1997.000000,0.260000,1.233389,0.000000,0.974387,0.000000,0.000097,0.000000,0.565035,801.488133
50%,2004.500000,0.560000,9.157214,0.124576,4.934715,0.230832,0.400382,0.000000,2.376270,3040.517926
75%,2012.000000,1.140000,58.731783,8.010227,28.476452,12.577810,1.877339,0.347518,6.495875,11253.254178
max,2019.000000,17.740000,10740.996069,7543.157408,1559.728160,825.415088,826.876048,62.654678,66.817851,123678.702100
